### Load Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import f1_score
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid
%matplotlib inline

### Load DataFrame

In [ ]:
df = pd.read_csv("/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")
pd.options.display.max_columns = 785

In [ ]:
df.head() # see top five records

### Seperate Input and Output Features

In [ ]:
df.rename(columns={'0':'label'}, inplace=True) # rename 0 column are label
X = df.drop('label',axis = 1).values # Input Features
y = df['label'].values # Target Variable

In [ ]:
labels = {i:chr(i+97) for i in range(26)}
print(labels)

### Train Test Split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.3)

### Convert Features to Torch Tensors

In [ ]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

### Create Model

In [ ]:
class Model(nn.Module):
    def __init__(self,input_features=784,hidden1=256,hidden2=128,hidden3=64,out_features=26):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.f_connected3=nn.Linear(hidden2,hidden3)
        self.out=nn.Linear(hidden3,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=F.relu(self.f_connected3(x))
        x=self.out(x)
        return x

In [ ]:
torch.manual_seed(20)
model= Model()

In [ ]:
model.parameters

### Define Loss Function and Optimizer

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

### Model Training

In [ ]:
epochs=200
losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    losses.append(loss)
    if i%10==0:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### Plot Loss vs Epochs

In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

### Validation Data Prediction

In [ ]:
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())

### Accuracy Score

In [ ]:
score=accuracy_score(y_test,predictions)
score